# Loading Data to Silver Zone

This Notebook:
* We will iIngest data from **Bronze Zone** to **Silver Zone** using spark
* We will use Spark Structured Streaming with **`trigger(availableNow=True)`** for batch loading
* We will do a **load control** of the batch processes through Structured Streaming **checkpoint**
* We will use **`awaitTermination()`**  ethod to transform the streaming queries in a synchronous process
* We will Combine spark and sql in order to do the data load

## 1.0 Initial Setup

In [0]:
%run "/Users/cabreirajm@gmail.com/DataPipelineCabreira/Helpers/data_generator" 


## 2.0 Create `Silver Zone` Schema

In [0]:
spark.sql("CREATE DATABASE IF NOT EXISTS silver")

DataFrame[]

## 3.0 Businesse Requirements for Silver Zone

1. The ingestion need to be done in batch in order to avoid extra costs 
    * Even though the API data is stored in streaming in landing zone 
2. API and Batch Data need to be stored in the same table 
3. Each table will have an uuid column with a hash to identify each register
4. We have to garantee the correct data type of all column 
5. We need to create better column names for each table 

## 4.0 Data Modeling

### 4.1 Courses Table  ( Domain Table)

The table `tb_courses` is a **domain table** and we will store all the available courses information ( the product ). Its information will be added manually.
* We will use the **md5()** function to create the **curso_uuuid** column by the course name 
* The column **data_carga** : Contains the processing date

In [0]:
spark.sql("""
  CREATE TABLE IF NOT EXISTS silver.tb_courses
  AS
    SELECT  
      md5('Data Pipeline with Databricks') AS course_uuid,
      'Data Pipeline with Databricks' AS course_name,
      'beginner' AS course_level,
      589.90 AS course_price,
      getdate() AS dt_load

      UNION

    SELECT
      md5('From your first data pipeline to a Data Lakehouse with Databricks') AS course_uuid,
      'From your first data pipeline to a Data Lakehouse with Databricks' AS course_name,
      'intermediate' as course_level,
      659.90 AS course_price,
      getdate() AS dt_load


      UNION

    SELECT
      md5('Building a Data Pipeline with Spark Structured Streaming') AS course_uuid,
      'Building a Data Pipeline with Spark Structured Streaming' as course_name,
      'advanced' as course_level,
      549.90 as course_price,
      getdate() as dt_load
"""
)


spark.sql('SELECT * FROM silver.tb_courses').display()

course_uuid,course_name,course_level,course_price,dt_load
fb95df132ca7f41d392bc98ccf0cfeb8,Data Pipeline with Databricks,beginner,589.90,2024-11-22T19:12:29.318Z
bda125b01c9596e123e5f9b3bf00f3a8,From your first data pipeline to a Data Lakehouse with Databricks,intermediate,659.90,2024-11-22T19:12:29.318Z
ff17869bc6f9d9865e0bf8133c4ce3c3,Building a Data Pipeline with Spark Structured Streaming,advanced,549.90,2024-11-22T19:12:29.318Z


In [0]:
#%fs rm -r dbfs:/user/hive/warehouse/silver.db/tb_courses

We will now create two streaming views called **`stream_temp_vw_api`**  and **`stream_temp_vw_files`** that will be used as source data for our loading process.

In [0]:
api_df = spark.readStream.table('bronze.api_data')
api_df.createOrReplaceTempView('stream_temp_vw_api')

files_df = spark.readStream.table('bronze.file_data')
files_df.createOrReplaceTempView('stream_temp_vw_files')

### 4.2 Access Table  

This table stores all the website access.

**`df_access`** : Dataframe that used to load data into **`tb_access`** table. This dataframe stores all information regarding the website visitors and its information comes from API. 

**Columns:**
* **`acesso_uuid` column** : Created with the **`md5()`** function  by **`concatenating`** the columns below:
  * **`access_date`** - After being converted to Timestamp
  * **`ip_address`**. - ip address of the computer 
  * **`access_point`** -Identify the access point as mobile or computer (**local_acesso**).
* **`usuario_uuid` column**: Created with the **`md5()`** function  by **`concatenating`** the columns below:
  * **`access_date`** - After being converted to Timestamp
  * **`payload.info_usuario.nome`** - name of the user
* **`data_carga` column**: The processing date of the register 

In [0]:
%sql
select * from stream_temp_vw_api limit 2

access_date,access_point,ip_address,payload,_rescued_data,source_file_name,processing_timestamp
2024-06-02T17:30:35.000Z,iphone,69.127.75.83,"List(null, null, null)",null,part-00002-6a513f62-6f11-4849-8da2-4a202cc4adb9-c000.json,2024-11-22T19:51:02.88Z
2024-06-02T18:03:31.000Z,safari,168.18.37.100,"List(null, null, List(usuario_aea7b8dddc@hotmail.com, SP, 36, Usuario aea7b8dddc, Cientista de Dados, F))",null,part-00002-6a513f62-6f11-4849-8da2-4a202cc4adb9-c000.json,2024-11-22T19:51:02.88Z


In [0]:
df_access = spark.sql(""" 
    SELECT 
      CAST( access_date AS TIMESTAMP) AS access_timestamp,
      ip_address AS access_ip_address,
      CASE WHEN access_point IN ('iphone','android') THEN 'Mobile' ELSE 'Computer' END AS local_access,
      md5(concat(
        CAST(access_date AS TIMESTAMP),
        ip_address,
        CASE WHEN access_point IN ('iphne', 'android') THEN 'mobile' ELSE 'computer' END
      )) AS access_uuid,
      md5(concat(
          CAST( access_date AS TIMESTAMP),
          payload.info_usuario.nome        
         )) AS user_uuid,
        payload.info_produto.product_uuid AS course_uuid,
      getdate() AS dt_load
    FROM stream_temp_vw_api
"""
)

df_access.limit(5).display()

access_timestamp,access_ip_address,local_access,access_uuid,user_uuid,course_uuid,dt_load
2024-06-02T17:30:35Z,69.127.75.83,Mobile,66bf173550bab1074ad6d545ff36aad2,null,null,2024-11-22T20:03:43.776Z
2024-06-02T18:03:31Z,168.18.37.100,Computer,562d7cf09fe6bc868808516857fb2e95,a3320d218294674fbf0c2e14b90671df,null,2024-11-22T20:03:43.776Z
2024-06-02T18:36:27Z,113.109.66.208,Mobile,971c6d9c866c5c6703bc2eda589d37b1,null,null,2024-11-22T20:03:43.776Z
2024-06-02T19:09:23Z,87.241.252.59,Computer,c4309fa8f4d737ecd65ba8c77447fc32,null,null,2024-11-22T20:03:43.776Z
2024-06-02T19:42:19Z,188.111.120.11,Computer,88e32230f15f37a57eb319995e07d0b4,null,null,2024-11-22T20:03:43.776Z


We have just read the stream data from our API. 

Important note about spark:
* The origin and destination of the should have the same caracteristics. In other words:
  * `Source` : Stream data and `Destination`: Stream data
  * `Source` : Batch data and `Destionation`: batch data

In other to overcome this issue, we will use `Spark Structured Streaming` to read the sterming data in batch.

We use `Spark Structured Streaming` to load micro batch of data.

`Spark Structured Streaming` :
* No need to manage a checkpoint table to identify data that have been loead
* The `Spark Structured Streaming` uses a **`checkpoint directory`** defined by the writeStream method. This checkpoint stores the last file/offset/row  that have been stored.This way, in case of failing the process, the spark will be able to garantee the **Stream Exactly-Once Semantics**. In other words, the checkpoint is responsible for controlling the load as it should be loaded.
* **trigger(availableNow == True)** : States spark to do the load in batch by using the Structured Streaming Process. That way, spark will ingest the data in micro-batches. After finishing all mapped data 


O Spark Structured Streaming permite o uso do tipo de **trigger availableNow**. Quando definido como **True** dentro do método **`trigger`** no método **`writeStream`** indicará ao Spark que **realize a carga de dados em Batch** usando o processo do Structured Streaming. O spark irá realizar a leitura de todos os registros disponíveis para carga e irá realizar a ingestão de todos esses dados em micro-batchs. Ao terminar a execução de todos os registros mapeados no início do processo de carga, o **Spark will stop the  Stream query automatically.**.  



* **`.writeStream`** : Stores the dataframe data into the **silver.tb_access** table which will be created on-the-fly through the **`.table()`** method 
* **`.outputMode('append')`**: States the the data will be appended in the destiny table
* **`option('CheckpointLocation', access_checkpoint_location)`**: Defines the diretory where the spark will use to control the streaming data and perform the `exactly-once delivery`
* **`.trigger(availableNow=True)`**: States that the writeStream process will be performed in batch.
* **`.awaitTermination()`**: Makes the stream query a synchronous process. Used when the **`availableNow`** parameter is equal to **`True`**


In [0]:
%fs rm -r dbfs:/user/hive/warehouse/silver.db/tb_access

res1: Boolean = true

In [0]:
access_checkpoint_location = 'dbfs:/user/hive/warehouse/silver.db/_checkpoint/api/tb_access'
(
    df_access.writeStream
        .format('delta')
        .outputMode('append')
        .option('CheckpointLocation', access_checkpoint_location)
        .trigger(availableNow = True)
        .table('silver.tb_access').awaitTermination()
)



spark.sql('SELECT * FROM silver.tb_access LIMIT 5').display()

access_timestamp,access_ip_address,local_access,access_uuid,user_uuid,course_uuid,dt_load
2024-06-02T17:30:35Z,69.127.75.83,Mobile,66bf173550bab1074ad6d545ff36aad2,null,null,2024-11-22T20:09:19.298Z
2024-06-02T18:03:31Z,168.18.37.100,Computer,562d7cf09fe6bc868808516857fb2e95,a3320d218294674fbf0c2e14b90671df,null,2024-11-22T20:09:19.298Z
2024-06-02T18:36:27Z,113.109.66.208,Mobile,971c6d9c866c5c6703bc2eda589d37b1,null,null,2024-11-22T20:09:19.298Z
2024-06-02T19:09:23Z,87.241.252.59,Computer,c4309fa8f4d737ecd65ba8c77447fc32,null,null,2024-11-22T20:09:19.298Z
2024-06-02T19:42:19Z,188.111.120.11,Computer,88e32230f15f37a57eb319995e07d0b4,null,null,2024-11-22T20:09:19.298Z


### 4.3 Users Table 

### 4.3.1 API users data

* **`df_users_api`**: contains all the user information from the API data. 
* **`user_uuid`** : We will use the **`md5()`** function to create the `user_uuid`. To do that, we will concat the columns below:
  * **`access_date`** - we will cast to timestamp
  * **`payload.info_usuario.nome`** 
* **`origin`**: Informs wheather the data is from API or file ( data vault modelling principle)
* **`dt_load`**: The process date

In [0]:
df_user_api = spark.sql("""
          SELECT 
            md5(concat(
            cast(access_date AS TIMESTAMP),
            payload.info_usuario.nome
            )) AS user_uuid,
            payload.info_usuario.nome AS user_name,
            payload.info_usuario.email AS user_email,
            CAST(payload.info_usuario.idade  AS INT) AS user_idade,
            payload.info_usuario.sexo as user_gender,
            payload.info_usuario.estado AS user_state,
            payload.info_usuario.profissao AS user_profession,
            CAST(NULL AS STRING) AS company,
            'API' as origin,
            getdate() as dt_load
            FROM stream_temp_vw_api
            WHERE payload.info_usuario IS NOT NULL
""")

df_user_api.limit(5).display()

user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,origin,dt_load
a3320d218294674fbf0c2e14b90671df,Usuario aea7b8dddc,usuario_aea7b8dddc@hotmail.com,36,F,SP,Cientista de Dados,null,API,2024-11-22T20:10:08.846Z
8f7e5643e974a7aa1e06fe6099f063f7,Usuario aac1431fa5,usuario_aac1431fa5@outlook.com,44,F,SC,Desenvolvedor de Sistemas,null,API,2024-11-22T20:10:08.846Z
bc1df61364155727147f5662dc2d68b8,Usuario dd72bfbd99,usuario_dd72bfbd99@hotmail.com,47,M,AM,Cientista de Dados,null,API,2024-11-22T20:10:08.846Z
0f71205c4c1df4db312847e9757d65fb,Usuario 7f204ad514,usuario_7f204ad514@hotmail.com,20,F,RJ,Desenvolvedor de Sistemas,null,API,2024-11-22T20:10:08.846Z
fd6871915eba8fb5a24afb32fa4379f8,Usuario 289cc961b9,usuario_289cc961b9@hotmail.com,26,F,DF,Analista de BI,null,API,2024-11-22T20:10:08.846Z


* **`option('mergeSchema',True)`** Indicates the process of **Schema Evolution**.

* **`.trigger(availableNow=True)`** States that the writeStream occours in  **batch**.

* **`.awaitTermination()`**  Makes the stream query to be a synchronous 


In [0]:
user_api_checkpoint_path = "dbfs:/user/hive/warehouse/silver.db/_checkpoint/api/tb_users"
(
  df_user_api.writeStream
        .format('delta')
        .outputMode('append')
        .option('CheckpointLocation',user_api_checkpoint_path)
        .option('mergeSchema', True)
        .trigger(availableNow=True)
        .table('silver.tb_users')
).awaitTermination()

spark.sql("SELECT * FROM silver.tb_users WHERE origin = 'API' LIMIT 5").display()

user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,origin,dt_load
a3320d218294674fbf0c2e14b90671df,Usuario aea7b8dddc,usuario_aea7b8dddc@hotmail.com,36,F,SP,Cientista de Dados,null,API,2024-11-22T20:15:28.024Z
8f7e5643e974a7aa1e06fe6099f063f7,Usuario aac1431fa5,usuario_aac1431fa5@outlook.com,44,F,SC,Desenvolvedor de Sistemas,null,API,2024-11-22T20:15:28.024Z
bc1df61364155727147f5662dc2d68b8,Usuario dd72bfbd99,usuario_dd72bfbd99@hotmail.com,47,M,AM,Cientista de Dados,null,API,2024-11-22T20:15:28.024Z
0f71205c4c1df4db312847e9757d65fb,Usuario 7f204ad514,usuario_7f204ad514@hotmail.com,20,F,RJ,Desenvolvedor de Sistemas,null,API,2024-11-22T20:15:28.024Z
fd6871915eba8fb5a24afb32fa4379f8,Usuario 289cc961b9,usuario_289cc961b9@hotmail.com,26,F,DF,Analista de BI,null,API,2024-11-22T20:15:28.024Z


### 4.3.2 Files users data


We will now create a **`df_users_file`** that will be used to ingest users data from all batch files.

* **`usuario_uuid`** : We will use the `md5()` function with concat to create the user uuid
  * **`nome_empresa`**.
  * **`nome_funcionario`**.
* **`origin`** :
* **`dt_loat`**:

In [0]:
df_users_file = spark.sql("""
      SELECT   md5(CONCAT(
            nome_empresa,
            nome_funcionario
        )) AS user_uuid,
        nome_funcionario AS user_name,
        email_functionario AS user_email,
        CAST(idade AS INT) AS user_idade,
        sexo AS user_gender,
        estado as user_state,
        profissao as user_profession,  
        nome_empresa AS company,
        'FILE'  as origin,
        getdate() as dt_load       
        FROM stream_temp_vw_files
""")

df_users_file.limit(5).display()

user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,origin,dt_load
c34c4b0abe94489a7913fa596d214837,Funcionario 1c64a078b0,funcionario_1c64a078b0@empresaa.com.br,38,F,PA,Analista de Negocio,Empresa A,FILE,2024-11-22T20:43:10.995Z
de1fc924f1f7bd83af85323c09052166,Funcionario 7651263aa5,funcionario_7651263aa5@empresaa.com.br,20,M,AP,Analista de BI,Empresa A,FILE,2024-11-22T20:43:10.995Z
5ad24ca0983441d38ce7985203c02652,Funcionario 801f18b55b,funcionario_801f18b55b@empresaa.com.br,45,M,TO,Analista de Dados,Empresa A,FILE,2024-11-22T20:43:10.995Z
12ea0258081123e5b0327e253e870d78,Funcionario 2a2b38c0ee,funcionario_2a2b38c0ee@empresaa.com.br,31,F,MA,Cientista de Dados,Empresa A,FILE,2024-11-22T20:43:10.995Z
ba03dcb4b259bb550915ec7119a50ab2,Funcionario 667f2be2e5,funcionario_667f2be2e5@empresaa.com.br,44,M,PI,Analista de BI,Empresa A,FILE,2024-11-22T20:43:10.995Z


In [0]:
user_files_checkpoint_files = 'dbfs:/user/hive/warehouse/silver.db/_checkpoint/files/tb_users'
(
df_users_file.writeStream
        .format('delta')
        .outputMode('append')
        .option('CheckpointLocation',user_files_checkpoint_files)
        .option('mergeSchema',True)
        .trigger(availableNow=True)
        .table('silver.tb_users')
).awaitTermination()

spark.sql("SELECT * FROM silver.tb_users WHERE origin = 'FILE' LIMIT 5").display()

user_uuid,user_name,user_email,user_idade,user_gender,user_state,user_profession,company,origin,dt_load
c34c4b0abe94489a7913fa596d214837,Funcionario 1c64a078b0,funcionario_1c64a078b0@empresaa.com.br,38,F,PA,Analista de Negocio,Empresa A,FILE,2024-11-22T20:49:59.032Z
de1fc924f1f7bd83af85323c09052166,Funcionario 7651263aa5,funcionario_7651263aa5@empresaa.com.br,20,M,AP,Analista de BI,Empresa A,FILE,2024-11-22T20:49:59.032Z
5ad24ca0983441d38ce7985203c02652,Funcionario 801f18b55b,funcionario_801f18b55b@empresaa.com.br,45,M,TO,Analista de Dados,Empresa A,FILE,2024-11-22T20:49:59.032Z
12ea0258081123e5b0327e253e870d78,Funcionario 2a2b38c0ee,funcionario_2a2b38c0ee@empresaa.com.br,31,F,MA,Cientista de Dados,Empresa A,FILE,2024-11-22T20:49:59.032Z
ba03dcb4b259bb550915ec7119a50ab2,Funcionario 667f2be2e5,funcionario_667f2be2e5@empresaa.com.br,44,M,PI,Analista de BI,Empresa A,FILE,2024-11-22T20:49:59.032Z


### 4.4 Sales Table 

In [0]:
stop_all_streams()
clean_up_landing_dir()

stop_all_streams-inicio-2024-11-22 20:51:25.932676
O stream display_query_7 fui finalizado com sucesso.
O stream display_query_4 fui finalizado com sucesso.
O stream display_query_3 fui finalizado com sucesso.
O stream display_query_2 fui finalizado com sucesso.
stop_all_streams-fim-2024-11-22 20:51:27.312878
              
clean_up_landing_dir-inicio-2024-11-22 20:51:27.312988
Todos os arquivos e diretórios dentro de 'dbfs:/FileStore/landing/' foram excluidos com sucesso.
clean_up_landing_dir-fim-2024-11-22 20:51:27.370540
              
